##  <p style="font--family:verdana " ><a href = 'https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/'>Source  </a></p>
##  <p style="font--family:verdana " ><a href = 'https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/'>xgboost complete tuning  </a></p>

In [2]:
import skopt

In [1]:
from pandas import Series, DataFrame 
import xgboost as xgb 
import lightgbm as lgb
from datetime import datetime 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [142]:
data=pd.read_csv('adult.csv',header=None) 

<IPython.core.display.Javascript object>

In [143]:
data.columns=['age','workclass','fnlwgt','education','education-num','marital_Status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','Income'] 

In [144]:
cat_col = data.select_dtypes('object').columns.tolist()#.remove('Income')

In [145]:
cat_col.remove('Income')

In [146]:
cat_col

['workclass',
 'education',
 'marital_Status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country']

In [110]:
data.occupation.value_counts()

 Prof-specialty       2032
 Exec-managerial      2020
 Craft-repair         2013
 Sales                1854
 Adm-clerical         1841
 Other-service        1628
 Machine-op-inspct    1020
 ?                     966
 Transport-moving      758
 Handlers-cleaners     702
 Tech-support          518
 Farming-fishing       496
 Protective-serv       334
 Priv-house-serv        93
 Armed-Forces            6
Name: occupation, dtype: int64

In [147]:
l=LabelEncoder() 
for c in data.columns:
    col_type = data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data[c] = Series(l.fit_transform(data[c]))
        #data[c] = data[c].astype('category')

In [148]:
Series(l.fit_transform(data["education"])).value_counts()

11    5283
15    3587
9     2670
12     934
8      679
1      637
7      534
0      456
5      309
14     258
6      242
2      224
10     181
4      176
3       79
13      32
dtype: int64

In [150]:
data.Income.value_counts() 

0    12435
1     3846
Name: Income, dtype: int64

In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             16281 non-null  int64
 1   workclass       16281 non-null  int32
 2   fnlwgt          16281 non-null  int64
 3   education       16281 non-null  int32
 4   education-num   16281 non-null  int64
 5   marital_Status  16281 non-null  int32
 6   occupation      16281 non-null  int32
 7   relationship    16281 non-null  int32
 8   race            16281 non-null  int32
 9   sex             16281 non-null  int32
 10  capital_gain    16281 non-null  int64
 11  capital_loss    16281 non-null  int64
 12  hours_per_week  16281 non-null  int64
 13  native_country  16281 non-null  int32
 14  Income          16281 non-null  int32
dtypes: int32(9), int64(6)
memory usage: 1.3 MB


In [ ]:
#but not one hot encoding with light GBM
one_hot_workclass=pd.get_dummies(data.workclass) 
one_hot_education=pd.get_dummies(data.education) 
one_hot_marital_Status=pd.get_dummies(data.marital_Status) 
one_hot_occupation=pd.get_dummies(data.occupation)
one_hot_relationship=pd.get_dummies(data.relationship) 
one_hot_race=pd.get_dummies(data.race) 
one_hot_sex=pd.get_dummies(data.sex) 
one_hot_native_country=pd.get_dummies(data.native_country) 

In [ ]:
data.drop(['workclass','education','marital_Status','occupation','relationship','race','sex','native_country'],axis=1,inplace=True) 

In [ ]:
data=pd.concat([data,one_hot_workclass,one_hot_education,one_hot_marital_Status,one_hot_occupation,one_hot_relationship,one_hot_race,one_hot_sex,one_hot_native_country],axis=1)

In [ ]:
 _, i = np.unique(data.columns, return_index=True) 
data=data.iloc[:, i] 

In [157]:
x=data.drop('Income',axis=1) 
y=data.Income 

In [158]:
y.isnull().sum()

0

In [159]:
y.fillna(y.mode()[0],inplace=True) 

In [160]:
#Now splitting our dataset into test and train 
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

In [82]:
#Applying xgboost
#label is used to define our outcome variable
dtrain=xgb.DMatrix(x_train, y_train)
dtest=xgb.DMatrix(x_test)

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.workclass, education, marital_Status, occupation, relationship, race, sex, native_country

In [ ]:
parameters={'max_depth':7, 'eta':1, 'silent':1,'objective':'binary:logistic','eval_metric':'auc','learning_rate':.05, 'tree_method':'hist'}

In [ ]:
#training our model 
num_round=50
from datetime import datetime 
start = datetime.now() 
xg=xgb.train(parameters,dtrain,num_round) 
stop = datetime.now()

In [ ]:
#Execution time of the model 
execution_time_xgb = stop-start 
execution_time_xgb

In [ ]:
ypred=xg.predict(dtest) 
ypred

In [ ]:
np.unique(ypred)

In [ ]:
for i in range(0,4885): 
    if ypred[i]>=0.5:    
        ypred[i]=1 
    else: 
        ypred[i]=0  

In [ ]:
accuracy_xgb = accuracy_score(y_test,ypred) 
f1_xgb = f1_score(y_test, ypred)

In [ ]:
accuracy_xgb

## light GBM

In [186]:
import lightgbm as lgb

In [194]:
labels = data.columns.tolist()#.remove('Income')

In [195]:
labels.remove('Income')

In [196]:
labels

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital_Status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country']

In [197]:
train_data=lgb.Dataset(x_train,label=y_train, feature_name=labels, categorical_feature= cat_col)
#param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param = {'objective':'binary'}
param['metric'] = ['auc', 'binary_logloss']

In [198]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11396 entries, 4006 to 13255
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             11396 non-null  int64
 1   workclass       11396 non-null  int32
 2   fnlwgt          11396 non-null  int64
 3   education       11396 non-null  int32
 4   education-num   11396 non-null  int64
 5   marital_Status  11396 non-null  int32
 6   occupation      11396 non-null  int32
 7   relationship    11396 non-null  int32
 8   race            11396 non-null  int32
 9   sex             11396 non-null  int32
 10  capital_gain    11396 non-null  int64
 11  capital_loss    11396 non-null  int64
 12  hours_per_week  11396 non-null  int64
 13  native_country  11396 non-null  int32
dtypes: int32(8), int64(6)
memory usage: 979.3 KB


In [199]:
#num_round=50
start=datetime.now()
#lgbm=lgb.train(param,train_data,num_round)
lgbm=lgb.train(param,train_data)
stop=datetime.now()

[LightGBM] [Info] Number of positive: 2644, number of negative: 8752
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 644
[LightGBM] [Info] Number of data points in the train set: 11396, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232011 -> initscore=-1.196989
[LightGBM] [Info] Start training from score -1.196989


In [200]:
#Execution time of the model
execution_time_lgbm = stop-start
execution_time_lgbm

datetime.timedelta(microseconds=101357)

In [201]:
#predicting on test set
ypred2=lgbm.predict(x_test)
ypred2[0:5]  # showing first 5 predictions

array([0.17263741, 0.42848479, 0.00608317, 0.38495955, 0.10561234])

In [202]:
for i in range(0,4885):
    if ypred2[i]>=.5:       # setting threshold to .5
        ypred2[i]=1
    else:  
        ypred2[i]=0

In [203]:
#calculating accuracy/f1
#f1_lgbm = accuracy_score(ypred2,y_test)
#f1_lgbm
print(classification_report(y_test, ypred2))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      3683
           1       0.77      0.65      0.71      1202

    accuracy                           0.87      4885
   macro avg       0.83      0.79      0.81      4885
weighted avg       0.86      0.87      0.86      4885



In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
#calculating roc_auc_score for xgboost
auc_xgb =  roc_auc_score(y_test,ypred)
auc_xgb

In [ ]:
auc_lgbm = roc_auc_score(y_test,ypred2)
comparison_dict = {'accuracy score':(f1_lgbm,f1_xgb),'auc score':(auc_lgbm,auc_xgb),'execution time':(execution_time_lgbm,execution_time_xgb)}

In [ ]:
comparison_dict

In [ ]:
#Creating a dataframe ‘comparison_df’ for comparing the performance of Lightgbm and xgb. 
comparison_df = DataFrame(comparison_dict) 
comparison_df.index= ['LightGBM','xgboost'] 
comparison_df

In [6]:
%run Neptune.py.ipynb

SSLError: SSL certificate validation failed. Set NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE environment variable to accept self-signed certificates.

SSLError: SSL certificate validation failed. Set NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE environment variable to accept self-signed certificates.